# CER Electricity Revised March 2012

## Import libaries

In [1]:
import os
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", context="paper") # imporved plt styles

from tqdm.notebook import tqdm as tqdm
#from wwo_hist import retrieve_hist_data

## Function definitions

In [2]:
from pandas.tseries.offsets import Day, Minute
def date_parser(d):
    day_code=int(d[:3])
    time_code=int(d[3:])
    return pd.datetime(2009,1,1) + Day(day_code - 1) + Minute(30) * (time_code - 1)

In [3]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [4]:
def save_cleaned_csv(data, file_path, **kwargs):
    data.to_csv(file_path,
                float_format="%.3f",
                #chunksize=1e5,
                **kwargs)

In [5]:
def dowload_weather_data(api_key,
                         frequency,
                         start_date,
                         end_date):
    location_list = ['Dublin']
    hist_weather_data = retrieve_hist_data(api_key,
                                           location_list,
                                           start_date,
                                           end_date,
                                           frequency,
                                           store_df = True)
    return hist_weather_data

In [6]:
def save_load_temp_csv(load_data, weather_data, file_path, **kwargs):
    re_idx = sorted(load_data.index.drop_duplicates())
    weather_ridx = weather_data.reindex(re_idx).ffill(limit=2)

    if weather_ridx.isnull().any().sum() != 0:
        raise ValueError("Missing weather data in given load period")

    load_data = pd.merge(load_data,
                         weather_ridx,
                         how='left',
                         left_index=True,
                         right_index=True)

    if load_data.isnull().any().sum() != 0:
        raise ValueError("Data has NaNs")

    load_data.to_csv(file_path,
                     float_format="%.3f",
                     **kwargs)

In [7]:
def gen_date_time(date, minute = 0):
    date_time=pd.to_datetime(date,
                             unit='h',
                             infer_datetime_format=True)
    date_time+=pd.to_timedelta(minute*30,
                               unit='m')
    return date_time

In [8]:
def interpolate_missing_values(df):
    it=tqdm(total=6)
    it.set_description('set date time index')
    df=df.set_index('date_time') # set date time index to interpolate on
    it.update()
    it.set_description('groupby id')
    df=df.groupby('id') # group by housholds
    it.update()
    it.set_description('resampling')
    df=df.resample('30T') # resample with freq = 30M
    it.update()
    df=df.asfreq() # Return the values at the new freq, essentially a reindex.
    it.update()
    it.set_description('interpolating')
    df=df.interpolate(method='linear', limit_direction='forward', limit=2) # remove nans by forward interpolation
    it.update()
    it.set_description('dropping id')
    df=df.drop('id',axis='columns') # drop id since interpolation turned it into garbage 
    it.update()
    return df#.reset_index()

In [9]:
#ref.: https://www.kaggle.com/avanwyk/encoding-cyclical-features-for-deep-learning
def encode(data, cycl_name):
    cycl = getattr(data.date_time.dt, cycl_name)
    cycl_max = cycl.max()
    data[cycl_name + '_sin'] = np.sin(2 * np.pi * cycl/cycl_max, dtype=np.float32)
    data[cycl_name + '_cos'] = np.cos(2 * np.pi * cycl/cycl_max, dtype=np.float32)
    return data

## Load data

In [10]:
data_path=os.path.join("../../data",'CER Electricity Revised March 2012')
files=[file for file in os.listdir(data_path) if file.startswith('File') and file.endswith('.txt')]
files

['File6.txt', 'File1.txt', 'File5.txt', 'File2.txt', 'File4.txt', 'File3.txt']

In [11]:
file_name=os.path.join(data_path, 'SME and Residential allocations.txt')
df_categories=pd.read_csv(file_name,
      dtype={"Id":'uint16',"Code":"uint8"},
      sep="\s+")
#data[f]=df
#categories = pd.Categorical.from_codes(
#    df_categories.Code - 1,
#    ['residential','enterprise','other']
#)
df_categories.columns = df_categories.columns.str.lower()
df_categories.head()

,id,code
0,1000,3
1,1001,3
2,1002,1
3,1003,1
4,1004,1


In [12]:
residential_ids=df_categories[df_categories.code == 1].id
residential_ids.size

4225

In [13]:
data=dict()
it=tqdm(files)
for f in it:
    it.set_description(f"reading file: {f}")

    file_name=os.path.join(data_path, f)
    df=pd.read_csv(file_name,
          header=None,
          names=["id","date_time","load"],
          dtype={"id":'uint16', "load": 'float32'},
          #index_col=["id","date_time"],
          #parse_dates=[1],
          #date_parser=date_parser,
          sep="\s+")
    
    # drop all *non residential* ids
    it.set_description("select residential ids")
    df=df.loc[df.id.isin(residential_ids)]

    # parse timestamp
    it.set_description("generating timestamps")
    # digit 1-3 (day 1 = 2019/1/1)
    day_code=df.date_time // 100 - 1
    # digit 4-5 (1 = 00:00:00 - 00:29:59)
    time_code=df.date_time % 100 - 1

    date=pd.to_datetime(day_code,
                        unit='D',
                        origin=pd.Timestamp('2009-01-01'),
                        infer_datetime_format=True)
    time_delta=pd.to_timedelta(time_code*30,
                unit='m')
    df.date_time=date+time_delta
    #df.date_time=(date.astype('int') // (10**9*60*60)).astype('uint32')
    #df.rename(columns={'date_time':'date'},inplace=True)
    #df['weekday']=date.dt.weekday
    
    # reduce mem footprint
    #it.set_description("reduce mem footprint")
    #df['minute']=time_code.astype('uint8')
    #df.id=df.id.astype('uint16')
    
    # replace all invalid (0) fileds with NaN
    #df.load.replace(0.0,np.nan,inplace=True)

    data[f]=df

del df,day_code,time_code,date,df_categories

## Prepare data

In [14]:
data=pd.concat(data.values())

## Review data metrics

In [15]:
data.head()

,id,date_time,load
0,6496,2009-07-16 09:30:00,2.958
1,6496,2009-07-16 10:00:00,1.443
2,6496,2009-07-16 10:30:00,0.131
3,6496,2009-07-16 11:30:00,0.144
4,6496,2009-07-16 12:00:00,0.208


In [16]:
data.dtypes

id                   uint16
date_time    datetime64[ns]
load                float32
dtype: object

In [17]:
len(data.id.unique())

4225

In [18]:
mem_usage(data)

'2280.17 MB'

In [19]:
data.load.isnull().sum()

0

In [20]:
data.load[data.load < 0.0].any()

False

In [21]:
data.load.describe()

count    1.086788e+08
mean     5.012664e-01
std      6.843614e-01
min      0.000000e+00
25%      1.190000e-01
50%      2.510000e-01
75%      5.790000e-01
max      1.711500e+01
Name: load, dtype: float64

## Date Cleaning

### Delete assets missing values

In [22]:
size_ids=data.groupby('id').size()

In [23]:
min_records=size_ids.max()

In [24]:
(size_ids != min_records).sum()

586

In [25]:
(size_ids != min_records).sum()/len(data.id.unique())

0.138698224852071

In [26]:
low_data_ids=size_ids[size_ids != min_records].index
low_data_ids

UInt64Index([1008, 1025, 1034, 1068, 1078, 1080, 1094, 1112, 1134, 1152,
             ...
             7337, 7351, 7353, 7359, 7398, 7405, 7409, 7429, 7430, 7440],
            dtype='uint64', name='id', length=586)

In [27]:
data=data[~data.id.isin(low_data_ids)]

In [28]:
mem_usage(data)

'1964.47 MB'

In [29]:
len(data.id.unique())

3639

### time change

 * `25. Okt 2009` - Sommerzeit endete: $+1$ Stunde
 * `28. Mär 2010` - Sommerzeit begann: $-1$ Stunde
 * `31. Okt 2010` - Sommerzeit endete: $+1$ Stunde

In [30]:
num_duplicates=data.groupby(['id', data.date_time.dt.date]).size()
num_duplicates.unique()

array([48, 50, 46])

In [31]:
num_duplicates[num_duplicates == 46].index.get_level_values(1).unique()

Index([2010-03-28], dtype='object', name='date_time')

In [32]:
num_duplicates[num_duplicates == 50].index.get_level_values(1).unique()

Index([2009-10-26, 2010-11-01], dtype='object', name='date_time')

#### Merge duplicated hours due to summer time end

In [33]:
data=data.groupby(['id','date_time']).sum().reset_index()
data

,id,date_time,load
0,1002,2009-07-14 00:00:00,0.362
1,1002,2009-07-14 00:30:00,0.064
2,1002,2009-07-14 01:00:00,0.119
3,1002,2009-07-14 01:30:00,0.023
4,1002,2009-07-14 02:00:00,0.140
...,...,...,...
93616909,7443,2010-12-31 21:30:00,0.486
93616910,7443,2010-12-31 22:00:00,0.318
93616911,7443,2010-12-31 22:30:00,0.332
93616912,7443,2010-12-31 23:00:00,0.357


#### Interpolate missing hour on winter time end

In [34]:
data = interpolate_missing_values(data).reset_index()
data

,id,date_time,load
0,1002,2009-07-14 00:00:00,0.362
1,1002,2009-07-14 00:30:00,0.064
2,1002,2009-07-14 01:00:00,0.119
3,1002,2009-07-14 01:30:00,0.023
4,1002,2009-07-14 02:00:00,0.140
...,...,...,...
93624187,7443,2010-12-31 21:30:00,0.486
93624188,7443,2010-12-31 22:00:00,0.318
93624189,7443,2010-12-31 22:30:00,0.332
93624190,7443,2010-12-31 23:00:00,0.357


In [35]:
# Did it work?
#data.index.duplicated().sum()

In [36]:
#num_duplicates=data.groupby(['id', data.date_time.dt.date]).size()
#num_duplicates.unique()

## Add Holiday indicator and weekday

In [37]:
start_date=data.date_time.dt.date.min()
end_date=data.date_time.dt.date.max()
start_date,end_date

(datetime.date(2009, 7, 14), datetime.date(2010, 12, 31))

In [38]:
from holidays import Ireland as holidays_ir

holidays = holidays_ir()

In [39]:
h = holidays[start_date:end_date]
is_holiday = np.uint8(np.isin(data.date_time.dt.date,h))

In [40]:
data['is_holiday']=is_holiday

In [41]:
data['weekday']=np.uint8(data.date_time.dt.weekday)

## Encode Cyclic features as sin and cos

In [42]:
#data = encode(data, 'hour')

In [43]:
#data = encode(data, 'minute')

In [44]:
#data = encode(data, 'day')

In [45]:
#data = encode(data, 'month')

In [46]:
#data.head(48).plot.scatter('hour_cos', 'hour_sin').set_aspect('equal')

In [47]:
#data.head(10).plot.scatter('minute_cos', 'minute_sin')#.set_aspect('equal')

In [48]:
#data.head(48*31).plot.scatter('day_cos', 'day_sin').set_aspect('equal')

In [49]:
#data.head(48*30*12).plot.scatter('month_cos', 'month_sin').set_aspect('equal')

In [50]:
data.head()

,id,date_time,load,is_holiday,weekday
0,1002,2009-07-14 00:00:00,0.362,0,1
1,1002,2009-07-14 00:30:00,0.064,0,1
2,1002,2009-07-14 01:00:00,0.119,0,1
3,1002,2009-07-14 01:30:00,0.023,0,1
4,1002,2009-07-14 02:00:00,0.140,0,1


In [51]:
data.dtypes

id                    uint64
date_time     datetime64[ns]
load                 float32
is_holiday             uint8
weekday                uint8
dtype: object

In [52]:
mem_usage(data)

'1964.31 MB'

## Split in Training and Test data

In [53]:
data.set_index('date_time', inplace=True)

### Reeserve ~10% of data for testing

In [54]:
months=len(pd.period_range(start_date,end_date,freq='M'))
0.10*months

1.8

In [55]:
split_date=end_date-int(np.round(0.10*months))*pd.offsets.MonthBegin()
split_date.to_pydatetime().date()

datetime.date(2010, 11, 1)

In [56]:
f'train data from {start_date} till {split_date - pd.offsets.Minute(30)}'

'train data from 2009-07-14 till 2010-10-31 23:30:00'

In [57]:
f'test data from {split_date} till {end_date}'

'test data from 2010-11-01 00:00:00 till 2010-12-31'

In [58]:
test_data_split_day_str=str(split_date)
train_data_split_day_str=str(split_date - pd.offsets.Minute(30))
test_data_split_day_str,train_data_split_day_str

('2010-11-01 00:00:00', '2010-10-31 23:30:00')

In [59]:
test_data = data.loc[test_data_split_day_str:]

In [60]:
train_data = data.loc[:train_data_split_day_str]

In [61]:
test_data.load.count()/data.load.count()

0.11380597014925373

In [62]:
train_data.index.min(),train_data.index.max()

(Timestamp('2009-07-14 00:00:00'), Timestamp('2010-10-31 23:30:00'))

In [63]:
test_data.index.min(),test_data.index.max()

(Timestamp('2010-11-01 00:00:00'), Timestamp('2010-12-31 23:30:00'))

In [64]:
if test_data.index.isin(train_data.index.unique()).any():
    raise Error("Information leak detected!")

### Are ids in both sets?

In [65]:
assert test_data.id.unique() in train_data.id.unique() and train_data.id.unique() in test_data.id.unique(), 'not all assets in both sets'

## Save as CSV

In [66]:
cleaned_data_path=os.path.join('../../paper_data')

In [67]:
for ds,dat in tqdm(zip(['train','test'],[train_data,test_data]),total=2):
    file_path=os.path.join(cleaned_data_path,f'{ds}.csv')
    os.makedirs(os.path.dirname(file_path),exist_ok=True)
    dat.to_csv(file_path,
               float_format="%.3f")

In [68]:
!head -n 2 "$cleaned_data_path/train.csv"; echo '...'; tail -n 1 "$cleaned_data_path/train.csv"

date_time,id,load,is_holiday,weekday
2009-07-14 00:00:00,1002,0.362,0,1
...
2010-10-31 23:30:00,7443,0.120,0,6


In [69]:
!head -n 2 "$cleaned_data_path/test.csv"; echo '...'; tail -n 1 "$cleaned_data_path/test.csv"

date_time,id,load,is_holiday,weekday
2010-11-01 00:00:00,1002,0.101,0,0
...
2010-12-31 23:30:00,7443,0.332,0,4


In [70]:
read_csv=pd.read_csv(
    os.path.join(cleaned_data_path,'train.csv'),
    parse_dates=['date_time'],
    infer_datetime_format=True,
    index_col=['date_time'],
    dtype={'id': 'uint16',
           'load': 'float32',
           'is_holiday': 'uint8'})

In [71]:
read_csv

,id,load,is_holiday,weekday
date_time,,,,
2009-07-14 00:00:00,1002,0.362,0,1
2009-07-14 00:30:00,1002,0.064,0,1
2009-07-14 01:00:00,1002,0.119,0,1
2009-07-14 01:30:00,1002,0.023,0,1
2009-07-14 02:00:00,1002,0.140,0,1
...,...,...,...,...
2010-10-31 21:30:00,7443,0.254,0,6
2010-10-31 22:00:00,7443,0.167,0,6
2010-10-31 22:30:00,7443,0.137,0,6


In [72]:
mem_usage(read_csv)

'1819.89 MB'

## Subset of data as CSV

In [73]:
np.random.seed(42)
subset_size=0.1

In [74]:
ids=test_data.id.unique()
np.random.shuffle(ids)
ids=ids[:int(subset_size * len(ids))]

In [75]:
train_data_subset=train_data[train_data.id.isin(ids)]
test_data_subset=test_data[test_data.id.isin(ids)]

In [76]:
train_data_subset.id.unique() in test_data_subset.id.unique()

True

In [77]:
cleaned_subset_data_path=os.path.join(cleaned_data_path,'mini')

In [78]:
for ds,dat in tqdm(zip(['train','test'],[train_data_subset,test_data_subset]),total=2):
    file_path=os.path.join(cleaned_subset_data_path,f'{ds}.csv')
    os.makedirs(os.path.dirname(file_path),exist_ok=True)
    dat.to_csv(file_path,
               float_format="%.3f")

In [81]:
pd.Series(ids).to_csv(os.path.join(cleaned_subset_data_path,'subset_id.csv'), float_format="%.3f")